In [1]:
import numpy as np
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib
import cv2
import random
import tensorflow as tf
from tensorflow.python.framework import ops
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers import regression
from tflearn.data_utils import to_categorical

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open('number_train.csv', 'r') as csv_file:
    result = csv.reader(csv_file)
    rows = []
    
    for row in result:
        rows.append(row)

In [3]:
train_data = [] # dữ liệu training
train_label = [] # label của chúng

for letter in rows:
    x = np.array([int(j) for j in letter[1:]])
    x = x.reshape(28, 28)
    train_data.append(x)
    train_label.append(int(letter[0]))

In [4]:
shuffle_order = list(range(len(train_label)))
random.shuffle(shuffle_order)

train_data = np.array(train_data)
train_label = np.array(train_label)

train_data = train_data[shuffle_order]
train_label = train_label[shuffle_order]

In [5]:
train_x = train_data[3000:]
train_y = train_label[3000:]

val_x = train_data[:3000]
val_y = train_label[:3000]

In [6]:
with open('number_test.csv', 'r') as csv_file:
    result = csv.reader(csv_file)
    rows = []
    
    for row in result:
        rows.append(row)

In [7]:
test_data = [] # dữ liệu training
test_label = [] # label của chúng

for letter in rows:
    x = np.array([int(j) for j in letter[1:]])
    x = x.reshape(28, 28)
    test_data.append(x)
    test_label.append(int(letter[0]))

In [8]:
shuffle_test_order = list(range(len(test_label)))
random.shuffle(shuffle_test_order)

test_data = np.array(test_data)
test_label = np.array(test_label)

test_data = test_data[shuffle_test_order]
test_label = test_label[shuffle_test_order]

In [9]:
test_x = test_data[:9000]
test_y = test_label[:9000]

In [10]:
BATCH_SIZE = 32
IMG_SIZE = 28
N_CLASSES = 10
LR = 0.001
N_EPOCHS = 50

In [11]:
ops.reset_default_graph()

network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1]) #1

network = conv_2d(network, 32, 3, activation='relu') #2
network = max_pool_2d(network, 2) #3

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)

network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)

network = fully_connected(network, 1024, activation='relu') #4
network = dropout(network, 0.8) #5

network = fully_connected(network, N_CLASSES, activation='softmax')#6
network = regression(network)

model = tflearn.DNN(network) #7

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
train_x = train_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
val_x = val_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_x = test_x.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [13]:
original_test_y = test_y

train_y = to_categorical(train_y, N_CLASSES)
val_y = to_categorical(val_y, N_CLASSES)
test_y = to_categorical(test_y, N_CLASSES)

In [14]:
model.fit(train_x, train_y, n_epoch=N_EPOCHS, validation_set=(val_x, val_y), show_metric=True)

Training Step: 44549  | total loss: 0.01313 | time: 22.766s
| Adam | epoch: 050 | loss: 0.01313 - acc: 0.9975 -- iter: 56960/57000
Training Step: 44550  | total loss: 0.01232 | time: 23.803s
| Adam | epoch: 050 | loss: 0.01232 - acc: 0.9978 | val_loss: 0.08744 - val_acc: 0.9890 -- iter: 57000/57000
--


In [18]:
model.save('./model/numberModel.tflearn')

INFO:tensorflow:C:\Users\tri.tranminh\model\numberModel.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [19]:
model.load('./model/numberModel.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\tri.tranminh\model\numberModel.tflearn


In [20]:
test_logits = model.predict(test_x)
test_logits = np.argmax(test_logits, axis=-1)

In [21]:
print(np.sum(test_logits == original_test_y) / len(test_logits))

0.9901111111111112


In [22]:
print(test_logits)

[2 5 6 ... 3 2 7]
